### Clusterize users and find most popular categories in each cluster

Builds a sparse matrix using amount of views in each category as a feature coordinate

In [1]:
import numpy as np
from classeslib.features import build_categories_features_matrix

In [2]:
clusters_users_count = 100000
from classeslib.statistics import StatisticsDumper
from classeslib import persistence_files

In [3]:
user_ids = StatisticsDumper.get_all_users_ids(persistence_files.public_train_statistics_db_file)
train_users_ids = np.random.choice(user_ids, clusters_users_count, replace=False)
#train_users_ids = user_ids
train_users_interests = StatisticsDumper.get_users_interests(
    train_users_ids,
    persistence_files.public_train_statistics_db_file)

In [4]:
(d1_level_statistics,
 d2_level_statistics,
 d3_level_statistics) = StatisticsDumper.restore_statistics(persistence_files.public_train_statistics_db_file)

d1_categories_sorted = sorted(d1_level_statistics.categories_statistics.keys(),
                       key = lambda c : d1_level_statistics.categories_statistics[c],
                       reverse=True)
d2_categories_sorted = sorted(d2_level_statistics.categories_statistics.keys(),
                       key = lambda c : d2_level_statistics.categories_statistics[c],
                       reverse=True)
d3_categories_sorted = sorted(d3_level_statistics.categories_statistics.keys(),
                       key = lambda c : d3_level_statistics.categories_statistics[c],
                       reverse=True)
print len(d3_categories_sorted)

924


In [5]:
train_features_matrix = build_categories_features_matrix(
    d1_categories_sorted, d2_categories_sorted, d3_categories_sorted, train_users_interests)

In [6]:
print len(train_users_interests)
print train_features_matrix.shape

100000
(100000, 1062)


In [21]:
clusters_count = 250

In [22]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=clusters_count).fit(train_features_matrix)

In [26]:
from sklearn.cluster import MiniBatchKMeans
clusterization = MiniBatchKMeans(init=svd.components_, n_clusters=clusters_count, n_init=2)
users_clusterized = clusterization.fit_predict(train_features_matrix)

In [27]:
print len(set(clusterization.labels_))

250


In [28]:
import pickle
with open(persistence_files.cluster_model_file, "w") as cluster_file:
    pickle.dump(clusterization, cluster_file)

In [29]:
with open(persistence_files.svd_model_file, "w") as pca_file:
    pickle.dump(svd, pca_file)

In [30]:
from classeslib.clusterization import UsersClaster

In [31]:
def calculate_clusters(users_ids, users_clusters_labels, clusters_model, users_interests):
    clusters = {}

    row = 0
    for user_id in users_ids:
        label = users_clusters_labels[row]
        center = clusters_model.cluster_centers_[label]
        row += 1
        if not clusters.has_key(label):
            clusters[label] = UsersClaster(label, center)
        clusters[label].add_user_d3_categories(users_interests[user_id])
        
    for cluster in clusters.values():
        cluster.d3_categories_sorted = sorted(
            cluster.d3_categories_popularity.keys(),
            key = lambda d3: cluster.d3_categories_popularity[d3],
            reverse = True)
        
    return clusters 

In [32]:
cluster_categories = calculate_clusters(train_users_ids, users_clusterized, clusterization, train_users_interests)
print len(cluster_categories)

250


Persist cluster categories info as well

In [33]:
import shelve
clusters_categories_db = shelve.open(persistence_files.clusters_categories_db_file)
for (label, cluster) in cluster_categories.items():
    clusters_categories_db[str(label)] = cluster
clusters_categories_db.close()

In [34]:
print persistence_files.clusters_categories_db_file
print cluster_categories.keys()

./db/clusters_categories.db
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 2

Check clusters categories

In [35]:
assert len(cluster_categories), "no clusters!"

for cluster in cluster_categories.values():
    message = "cluster " + str(cluster.label)
    assert len(cluster.d3_categories_sorted) == len(cluster.d3_categories_popularity), message 
    most_popular = cluster.d3_categories_sorted[0]
    least_popular = cluster.d3_categories_sorted[len(cluster.d3_categories_sorted)-1]
    assert cluster.d3_categories_popularity[most_popular] >= cluster.d3_categories_popularity[least_popular], message
    
print "OK"

OK
